In [2]:
# Celda 1: Importaciones, Carga y Preparación SIMPLIFICADA de Datos

import os
import re
import pandas as pd
import faiss
import psutil
from dotenv import load_dotenv
from tqdm.auto import tqdm
import gradio as gr
from IPython.display import display
import unicodedata

# LlamaIndex Core y componentes
from llama_index.core import (
    VectorStoreIndex, StorageContext, Settings, Document,
    load_index_from_storage
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.prompts import PromptTemplate
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter
from llama_index.llms.google_genai import GoogleGenAI

# --- 1. Configurar Entorno ---
num_cores = psutil.cpu_count(logical=True)
os.environ['OMP_NUM_THREADS'] = str(num_cores)
os.environ['MKL_NUM_THREADS'] = str(num_cores)
print(f"Librerías importadas. ¡Entorno listo con {num_cores} núcleos disponibles!")

# --- 2. Funciones de Preparación de Datos ---
# ¡ACTUALIZADO! Apuntamos al nuevo archivo pre-procesado.
METADATA_FILE = os.path.join("data", "metadatos_con_autoria.csv") 

def normalize_text(text):
    """Convierte texto a minúsculas y elimina acentos."""
    if not isinstance(text, str):
        return ""
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

def load_and_prepare_metadata(filepath):
    """
    Carga los metadatos pre-procesados y solo crea la columna 'Autoría_norm'.
    Toda la lógica compleja de unión de autores ya no es necesaria.
    """
    try:
        df = pd.read_csv(filepath)
        # Una buena práctica es seguir rellenando nulos por si acaso.
        df.fillna("No especificado", inplace=True)
        print("Metadatos cargados y procesados desde el archivo limpio.")

        # --- LÓGICA SIMPLIFICADA ---
        # Ahora solo necesitamos crear la versión normalizada de tu columna 'Autoría' ya limpia.
        print("Creando columna 'Autoría_norm' desde la columna 'Autoría'...")
        
        # Se asume que la columna 'Autoría' ya existe y está limpia en el CSV.
        if 'Autoría' not in df.columns:
            raise ValueError("El archivo CSV debe contener una columna llamada 'Autoría'.")
            
        df['Autoría_norm'] = df['Autoría'].apply(normalize_text)
        
        print("Columna 'Autoría_norm' creada exitosamente.")
        
        # Mostramos las dos columnas relevantes para verificar.
        display(df[['Autoría', 'Autoría_norm']].head())
        return df

    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo de metadatos en la ruta: {filepath}")
        return None
    except Exception as e:
        print(f"ERROR al cargar o procesar metadatos: {e}.")
        return None

# --- Ejecución ---
# Cargar y preparar el DataFrame en un solo paso.
df_metadatos = load_and_prepare_metadata(METADATA_FILE)

Librerías importadas. ¡Entorno listo con 6 núcleos disponibles!
Metadatos cargados y procesados desde el archivo limpio.
Creando columna 'Autoría_norm' desde la columna 'Autoría'...
Columna 'Autoría_norm' creada exitosamente.


,Autoría,Autoría_norm
0,"Bravo, David; Ruiz-Tagle, Jaime; Sanhueza, Ric...","bravo, david; ruiz-tagle, jaime; sanhueza, ric..."
1,"Raczynski, Dagmar; Pavez, M. Angélica","raczynski, dagmar; pavez, m. angelica"
2,"Marshall, Guillermo; Correa, Lorena","marshall, guillermo; correa, lorena"
3,"Marshall, Guillermo; Correa, Lorena","marshall, guillermo; correa, lorena"
4,"Fernández, Carolina; Jashes, Jessana","fernandez, carolina; jashes, jessana"


In [3]:
# Celda 2: CONFIGURACIÓN OPTIMIZADA DE MODELOS DE IA

# Cargar la clave de API desde el archivo .env
load_dotenv()
if "OPENROUTER_API_KEY" in os.environ:
    print("Clave de API de OpenRouter cargada.")
else:
    print("¡ADVERTENCIA! No se encontró la clave de API de OpenRouter.")

if "HF_TOKEN" in os.environ:
    print("Token de Hugging Face (HF_TOKEN) encontrado en el entorno.")
else:
    print("¡ADVERTENCIA! No se encontró el token de Hugging Face.")

# --- Configuración global de LlamaIndex optimizada para CPU ---
optimal_batch_size = max(64, num_cores * 8)
print(f"Configurando el modelo de embeddings con batch size óptimo de {optimal_batch_size} para {num_cores} núcleos.")

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
    embed_batch_size=optimal_batch_size,
    cache_folder="./model_cache",
    normalize=True,
    trust_remote_code=True
)

# Selecciona el proveedor de LLM aquí: "openrouter" o "gemini"
LLM_PROVIDER = "gemini"

if LLM_PROVIDER == "gemini":
    Settings.llm = GoogleGenAI(
        api_key=os.getenv("GOOGLE_API_KEY"),
        model="gemini-1.5-flash-latest",
        temperature=0.1,
    )
    print("Usando Gemini como modelo LLM.")
else:
    Settings.llm = OpenRouter(
        model="google/gemma-3n-e4b-it:free",
        #model="google/gemma-3n-e2b-it:free",
        #model="mistralai/mistral-small-3.2-24b-instruct:free",
        #model="deepseek/deepseek-chat-v3-0324:free",
        #model="deepseek/deepseek-r1-0528:free",
        #model="deepseek/deepseek-chat:free",
        #model="google/gemini-2.0-flash-exp:free",
        #model="mistralai/mistral-nemo:free",
        #model="qwen/qwq-32b:free",
        #model="microsoft/mai-ds-r1:free",
        #model="meta-llama/llama-4-maverick:free",
        temperature=0.1, 
    )
    print("Usando OpenRouter como modelo LLM.")

# Función para monitorear uso de memoria
def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Uso de memoria actual: {memory_info.rss / (1024 * 1024):.2f} MB")

print("\nConfiguración de modelos completada.")
print_memory_usage()

Clave de API de OpenRouter cargada.
Token de Hugging Face (HF_TOKEN) encontrado en el entorno.
Configurando el modelo de embeddings con batch size óptimo de 64 para 6 núcleos.
Usando Gemini como modelo LLM.

Configuración de modelos completada.
Uso de memoria actual: 553.27 MB


In [4]:
# Celda 3: Creación o Carga PERSISTENTE del Índice Vectorial (Optimizada para Hugging Face Spaces)

# 1. Definimos las rutas donde se guardarán los componentes del índice
PERSIST_DIR = "./storage_index"
FAISS_INDEX_PATH = os.path.join(PERSIST_DIR, "faiss_index.bin")
DOCSTORE_PATH = os.path.join(PERSIST_DIR, "docstore.json")
INDEX_STORE_PATH = os.path.join(PERSIST_DIR, "index_store.json")

# Función para eliminar una carpeta y su contenido de forma segura
def remove_directory_safely(directory_path):
    import shutil
    try:
        if os.path.exists(directory_path):
            print(f"Eliminando directorio '{directory_path}'...")
            shutil.rmtree(directory_path)
            print(f"Directorio '{directory_path}' eliminado exitosamente.")
        return True
    except Exception as e:
        print(f"Error al eliminar directorio: {e}")
        return False

try:
    # Verificamos si existe el índice FAISS serializado
    if os.path.exists(FAISS_INDEX_PATH) and os.path.exists(DOCSTORE_PATH) and os.path.exists(INDEX_STORE_PATH):
        print(f"Cargando componentes del índice desde '{PERSIST_DIR}'...")
        try:
            # 1. Cargamos el índice FAISS con su método nativo
            faiss_index = faiss.read_index(FAISS_INDEX_PATH)
            print("Índice FAISS cargado correctamente.")
            
            # 2. Creamos el vector store y el storage context
            vector_store = FaissVectorStore(faiss_index=faiss_index)
            storage_context = StorageContext.from_defaults(
                vector_store=vector_store,
                persist_dir=PERSIST_DIR
            )
            
            # 3. Cargamos el índice completo
            metadata_index = load_index_from_storage(storage_context)
            print("¡Índice completo cargado exitosamente!")
            
        except Exception as e:
            print(f"Error al cargar algún componente del índice: {e}")
            if remove_directory_safely(PERSIST_DIR):
                print("Recreando el índice desde cero...")
            else:
                raise Exception("No se pudo eliminar el índice corrupto. Elimínalo manualmente.")
    
    # Si llegamos aquí, es porque necesitamos crear el índice (la carpeta no existe o se eliminó)
    if not os.path.exists(PERSIST_DIR) or not os.path.exists(FAISS_INDEX_PATH):
        print("Creando un nuevo índice...")
        
        if df_metadatos is None:
            print("No se puede crear el índice porque el DataFrame de metadatos no se cargó. Revisa la Celda 1.")
            metadata_index = None
        else:
            # --- NUEVO Y CORRECTO BLOQUE PARA PEGAR EN CELDA 3 ---

            # --- Lógica de Creación de Índice Optimizada ---
            print("="*50)
            print("INICIANDO PROCESO DE CREACIÓN DE ÍNDICE (LENTO LA PRIMERA VEZ)...\nEste proceso solo se ejecutará una vez.")
            print("="*50)

            Settings.node_parser = SentenceSplitter(chunk_size=2048)

            def optimize_text_for_embedding(text, max_length=1500):
                if not isinstance(text, str): return ""
                text = text[:max_length]
                text = re.sub(r'\s+', ' ', text)
                return text.strip()
            
            # --- Definición de la estrategia de contenido ---
            
            # Columnas con texto rico para la BÚSQUEDA SEMÁNTICA.
            columnas_contenido_semantico = [
                'Título del estudio', 'Resumen', 'Palabras clave', 'Objetivo', 'Metodología'
            ]
            # Metadatos categóricos que queremos que sean "buscables".
            columnas_metadatos_buscables = [
                'Categoría', 'Tipo de documento', 'Idioma', 'Incorpora perspectiva de género', 'Entidad a cargo del estudio'
            ]
            # Columnas que queremos GUARDAR para MOSTRAR en los resultados finales.
            columnas_para_metadatos = list(set(
                columnas_contenido_semantico + columnas_metadatos_buscables + 
                ['Autoría', 'Año de publicación', 'Url']
            ))

            print("Convirtiendo metadatos en documentos (contenido vs. metadatos)...")
            metadata_documents = []
            for index, row in tqdm(df_metadatos.iterrows(), total=df_metadatos.shape[0], desc="Procesando metadatos"):
                
                partes_texto_buscable = []
                # Añadir contenido semántico principal
                for col in columnas_contenido_semantico:
                    valor = row.get(col)
                    if pd.notna(valor) and str(valor) not in ["No especificado", ""]:
                        texto_optimizado = optimize_text_for_embedding(str(valor))
                        repeticiones = 2 if col in ['Título del estudio', 'Resumen'] else 1
                        for _ in range(repeticiones):
                           partes_texto_buscable.append(f"{col.replace('_', ' ')}: {texto_optimizado}")
                
                # Añadir metadatos clave como frases para mejorar la búsqueda
                for col in columnas_metadatos_buscables:
                    valor = row.get(col)
                    if pd.notna(valor) and str(valor) not in ["No especificado", ""]:
                        partes_texto_buscable.append(f"Información de {col.replace('_', ' ')}: {str(valor)}.")

                contenido_buscable = ". ".join(partes_texto_buscable)
                
                # Preparar el diccionario de METADATOS completo para mostrar
                metadatos_para_nodo = {
                    col: row.get(col, "No especificado") for col in columnas_para_metadatos
                }

                # Crear el Documento con la separación clara de responsabilidades
                doc = Document(
                    text=contenido_buscable,
                    metadata=metadatos_para_nodo
                )
                metadata_documents.append(doc)
            
            print("Creando índice vectorial con FAISS...")
            d = len(Settings.embed_model.get_text_embedding("test"))
            faiss_index = faiss.IndexFlatL2(d)
            vector_store = FaissVectorStore(faiss_index=faiss_index)
            storage_context = StorageContext.from_defaults(vector_store=vector_store)
            
            metadata_index = VectorStoreIndex.from_documents(
                metadata_documents,
                storage_context=storage_context,
                show_progress=True 
            )
            
            # --- ¡PASOS CRUCIALES DE GUARDADO! ---
            # 1. Creamos el directorio si no existe
            os.makedirs(PERSIST_DIR, exist_ok=True)
            
            # 2. Guardamos el índice FAISS usando su método nativo
            print(f"Guardando el índice FAISS en '{FAISS_INDEX_PATH}'...")
            faiss.write_index(faiss_index, FAISS_INDEX_PATH)
            
            # 3. Guardamos el resto del storage context (docstore, index_store)
            print(f"Guardando los componentes de LlamaIndex en '{PERSIST_DIR}'...")
            metadata_index.storage_context.persist(persist_dir=PERSIST_DIR)
            
            print("="*50)
            print("¡ÍNDICE CREADO Y GUARDADO EN FORMATO COMPATIBLE!")
            print("="*50)

except Exception as e:
    print(f"Error no manejado: {e}")
    print("Por favor, borra manualmente la carpeta 'storage_index' e intenta nuevamente.")
    metadata_index = None

Creando un nuevo índice...
INICIANDO PROCESO DE CREACIÓN DE ÍNDICE (LENTO LA PRIMERA VEZ)...
Este proceso solo se ejecutará una vez.
Convirtiendo metadatos en documentos (contenido vs. metadatos)...


Procesando metadatos:   0%|          | 0/837 [00:00<?, ?it/s]

Creando índice vectorial con FAISS...


Parsing nodes:   0%|          | 0/837 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/841 [00:00<?, ?it/s]

Guardando el índice FAISS en './storage_index\faiss_index.bin'...
Guardando los componentes de LlamaIndex en './storage_index'...
¡ÍNDICE CREADO Y GUARDADO EN FORMATO COMPATIBLE!


In [ ]:
# Celda 4: Lógica del Chatbot e Interfaz Gradio (VERSIÓN DEFINITIVA Y CORREGIDA)

if 'metadata_index' not in locals() or metadata_index is None or 'df_metadatos' not in locals() or df_metadatos is None:
    print("ERROR: El índice o el DataFrame no han sido creados. Por favor, ejecuta las celdas anteriores correctamente.")
else:
    # --- FUNCIONES DE APOYO ---
    def normalize_text(text):
        if not isinstance(text, str): return ""
        return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn').lower()

    def get_initial_chat_state():
        return {"user_name": None, "current_mode": "waiting_for_name", "temp_author": None,
                "last_results": [], "current_page": 0, "last_search_type": None}
    
    def search_by_topic(message, author_filter=None, k=50):
        print(f"Búsqueda semántica por tema: '{message}' con filtro de autor: {author_filter}, k={k}")
        
        # --- PROMPT TEMPLATE REESCRITO PARA MÁXIMA CLARIDAD Y SIN ERRORES ---
        prompt_text = f"""Considerando que la búsqueda es en una base de datos de documentos del Ministerio de Educación de Chile, que van de tematicas desde educación parvularia, presupuestos, educación básica, media, superior, educación de adultos, educación técnica y profesional, asistencia a clases, ayudas estudiantiles, junji, junaeb, etc. reescribe la consulta del usuario en una lista de 5 a 10 términos de búsqueda semánticamente relacionados y concisos, separados por comas. Responde SÓLO con los términos.

--- EJEMPLO ---
Consulta: 'asistencia'
Términos: asistencia, asistencia escolar, inasistencia, ausentismo escolar, tasa de asistencia, ayuda, subvenciones, auxilio
--- FIN EJEMPLO ---

Consulta: '{message}'
Términos:"""
        extraction_template = PromptTemplate(prompt_text)

        response = Settings.llm.complete(extraction_template.format(query_str=message))
        llm_term = str(response).strip().lower().replace('*', '').replace('-', '')
        print(f"Término de búsqueda final: '{llm_term}'")
        
        final_query_for_retriever = f"{message}. {llm_term}"
        
        retriever = metadata_index.as_retriever(similarity_top_k=k)
        retrieved_nodes = retriever.retrieve(final_query_for_retriever)
        results = []
        
        if retrieved_nodes:
            boost_terms = {term.strip() for term in normalize_text(message).split()}
            processed_results = []
            for node_with_score in retrieved_nodes:
                meta = node_with_score.node.metadata
                texto_relevante_norm = normalize_text(f"{meta.get('Título del estudio', '')} {meta.get('Resumen', '')} {meta.get('Palabras clave', '')}")
                bonus = sum(0.1 for term in boost_terms if term in texto_relevante_norm)
                processed_results.append({'node_with_score': node_with_score, 'boosted_score': node_with_score.score + bonus})

            processed_results.sort(key=lambda x: x['boosted_score'], reverse=True)

            normalized_author_filter_parts = normalize_text(author_filter).split() if author_filter else []
            all_search_terms = {term.strip() for term in final_query_for_retriever.replace('.',',').split(',') if term.strip()}

            for item in processed_results:
                node_with_score = item['node_with_score']
                meta = node_with_score.node.metadata
                
                if normalized_author_filter_parts:
                    autoria_en_metadata = meta.get('Autoría', '')
                    autoria_normalizada = normalize_text(autoria_en_metadata)
                    if not all(part in autoria_normalizada for part in normalized_author_filter_parts):
                        continue

                texto_relevante_norm_pistas = normalize_text(f"{meta.get('Título del estudio', '')} {meta.get('Resumen', '')} {meta.get('Palabras clave', '')}")
                motivos_encontrados = {term for term in all_search_terms if term in texto_relevante_norm_pistas}
                motivo_final = ""
                if motivos_encontrados:
                    motivo_final = ", ".join(sorted(list(motivos_encontrados)))
                else:
                    palabras_clave_doc = meta.get('Palabras clave', '')
                    if palabras_clave_doc and palabras_clave_doc != "No especificado":
                        motivo_final = f"Conceptos del documento: {palabras_clave_doc}"

                results.append({
                    'Título del estudio': meta.get('Título del estudio', 'N/A'), 
                    'Autor(es)': meta.get('Autoría', 'Sin información'), 
                    'Año de publicación': meta.get('Año de publicación', 'N/A'), 
                    'Url': meta.get('Url', 'No encontrado'),
                    '__is_semantic': True, 
                    'score': node_with_score.score,
                    'pistas_relevancia': motivo_final
                })

        relevant_results = [res for res in results if res["score"] >= 0.40]
        return {"results": relevant_results, "original_query": message, "expanded_query": llm_term}
    
    def search_by_author(authors_query):
        print(f"Búsqueda por autor/es: '{authors_query}'.")
        author_groups = [group.strip() for group in authors_query.split(',') if group.strip()]
        
        def df_to_results_list(df):
            records = []
            for _, row in df.iterrows():
                records.append({
                    'Título del estudio': row.get('Título del estudio', 'N/A'),
                    'Autor(es)': row.get('Autoría', 'Sin información'),
                    'Año de publicación': row.get('Año de publicación', 'N/A'),
                    'Url': row.get('Url', 'No encontrado'),
                    'score': 0, 'pistas_relevancia': '', '__is_semantic': False 
                })
            return records

        and_mask = pd.Series([True] * len(df_metadatos), index=df_metadatos.index)
        for group in author_groups:
            normalized_group_parts = normalize_text(group).split()
            for part in normalized_group_parts:
                and_mask &= df_metadatos['Autoría_norm'].str.contains(part, na=False)
        
        and_hits_df = df_metadatos[and_mask].copy()

        if not and_hits_df.empty:
            and_hits_df['Año de publicación num'] = pd.to_numeric(and_hits_df['Año de publicación'], errors='coerce').fillna(0)
            sorted_df = and_hits_df.sort_values(by='Año de publicación num', ascending=False)
            results_list = df_to_results_list(sorted_df)
            header = f"He encontrado {len(results_list)} documento(s) que incluyen a '{authors_query}'."
            return {"results": results_list, "header": header}

        if len(author_groups) > 1:
            response_parts = [f"No he encontrado documentos que incluyan a todos estos autores juntos. A continuación, los resultados para cada uno por separado:\n"]
            any_found = False
            for group in author_groups:
                group_mask = pd.Series([True] * len(df_metadatos), index=df_metadatos.index)
                normalized_group_parts = normalize_text(group).split()
                for part in normalized_group_parts:
                    group_mask &= df_metadatos['Autoría_norm'].str.contains(part, na=False)
                individual_hits_df = df_metadatos[group_mask].copy()
                if not individual_hits_df.empty:
                    any_found = True
                    total_individual = len(individual_hits_df)
                    limit_text = f"(mostrando los 5 más recientes de {total_individual})" if total_individual > 5 else ""
                    response_parts.append(f"\n---\n\n### Documentos de '{group}' {limit_text}:\n")
                    individual_hits_df['Año de publicación num'] = pd.to_numeric(individual_hits_df['Año de publicación'], errors='coerce').fillna(0)
                    sorted_individual_df = individual_hits_df.sort_values(by='Año de publicación num', ascending=False).head(5)
                    for i, (_, row) in enumerate(sorted_individual_df.iterrows(), 1):
                        response_parts.append(f"{i}. **Título:** {row.get('Título del estudio', 'N/A')}\n   - **Autor(es):** {row.get('Autoría', 'Sin información')}\n   - **Año:** {row.get('Año de publicación', 'N/A')}\n   - **URL:** {row.get('Url', 'No disponible')}\n")
                else:
                    response_parts.append(f"\n---\n\n### Documentos de '{group}':\n\nNo se encontraron documentos.")
            if any_found:
                return {"results": [], "header": "\n".join(response_parts), "is_special_format": True}
        return {"results": [], "header": f"No he encontrado documentos de '{authors_query}'. Revisa la ortografía."}
    
    def format_results_page(state):
        total_found = len(state["last_results"])
        if total_found == 0:
            return state.get("custom_header") or "No se encontraron resultados que coincidan con tu búsqueda."

        MAX_TO_SHOW = 5
        start_index = (state["current_page"] - 1) * MAX_TO_SHOW
        nodes_to_show = state["last_results"][start_index : start_index + MAX_TO_SHOW]

        if not nodes_to_show:
            return "No hay más documentos que mostrar. Ya estás en la última página."

        header_parts = []
        if state.get("custom_header"):
            header_parts.append(state.get("custom_header"))
        elif state.get("last_search_type") == "topic" and state.get("expanded_query"):
            header_parts.append(f"Para tu búsqueda sobre **'{state['original_query']}'**, he consultado los términos: *`{state['expanded_query']}`*.")
        
        if total_found > 1:
            page_info = f"(página {state['current_page']} de {-(-total_found // MAX_TO_SHOW)}, mostrando {len(nodes_to_show)} de {total_found} en total)"
            header_parts.append(page_info)
        
        header = " ".join(header_parts) + ":\n\n"

        lista_formateada = []
        for i, res in enumerate(nodes_to_show, start=start_index + 1):
            titulo = res.get('Título del estudio', 'N/A')
            autores = res.get('Autor(es)', 'N/A')
            ano = str(res.get('Año de publicación', 'N/A'))
            url = res.get('Url', 'No encontrado')
            
            relevance_str = ""
            if res.get('__is_semantic'):
                score = res.get('score', 0)
                if score > 0:
                    relevance_label = "Muy Alta" if score >= 0.75 else "Alta" if score >= 0.60 else "Media"
                    relevance_str = f"**(Relevancia: {relevance_label} [{score:.0%}])**"
            
            pistas_str = ""
            if res.get('pistas_relevancia'):
                pistas_str = f"\n   - *Pistas de Relevancia: {res.get('pistas_relevancia')}*"
            
            lista_formateada.append(f"{i}. **Título:** {titulo} {relevance_str}\n   - **Autor(es):** {autores}\n   - **Año:** {ano}\n   - **URL:** {url}{pistas_str}")

        footer = f"\n\n--- \n*Para ver más, escribe **siguiente** o `página {state['current_page'] + 1}`.*" if (start_index + MAX_TO_SHOW) < total_found else ""
        return header + "\n\n".join(lista_formateada) + footer
    
    def extract_name_from_greeting(message):
        patterns = [r"(?:soy|me llamo|mi nombre es|ll[aá]mame|dime)\s+(.+)", r"^(?:hola,? soy|hola,? mi nombre es)\s+(.+)"]
        for pattern in patterns:
            match = re.search(pattern, message, re.IGNORECASE)
            if match:
                return match.group(1).strip().title()
        return message.strip().title()

    # --- INTERFAZ GRADIO ---
    with gr.Blocks(theme="default") as demo:
        chat_state = gr.State(get_initial_chat_state())
        gr.Markdown("# CemIA - Asistente de IA del Centro de Estudios del MINEDUC")
        chatbot = gr.Chatbot(
            value=[{"role": "assistant", "content": "¡Hola! Soy CemIA, asistente de IA del Centro de Estudios del MINEDUC. ¿Cómo quieres que te llame?"}],
            type="messages", 
            label="CemIA", 
            height=500
        )
        
        with gr.Row(visible=False) as button_row:
            # CORRECCIÓN: Se separaron los botones para mayor claridad.
            btn_author = gr.Button("Búsqueda por autoría")
            btn_topic = gr.Button("Búsqueda por temática")
            btn_combo = gr.Button("Búsqueda por autoría y temática")
            btn_bye = gr.Button("Adiós")
            
        user_input = gr.Textbox(placeholder="Escribe tu nombre para comenzar...", label="Tu respuesta", autofocus=True)

        def make_buttons_interactive(interactive=True):
            return gr.update(interactive=interactive), gr.update(interactive=interactive), gr.update(interactive=interactive), gr.update(interactive=interactive)

        def handle_user_input(message, history, current_state):
            history.append({"role": "user", "content": message})
            mode = current_state["current_mode"]
            
            message_lower = message.strip().lower()
            if message_lower in ["siguiente", "más", "mas"] or re.search(r"p[aá]gina\s+(\d+)", message_lower):
                if current_state["last_results"]:
                    page_match = re.search(r"p[aá]gina\s+(\d+)", message_lower)
                    current_state["current_page"] = int(page_match.group(1)) if page_match else current_state["current_page"] + 1
                    history.append({"role": "assistant", "content": format_results_page(current_state)})
                else:
                    history.append({"role": "assistant", "content": "No hay resultados anteriores para paginar. Por favor, realiza una nueva búsqueda."})
                return history, current_state, gr.update(), gr.update(), gr.update(), gr.update(), gr.update(), gr.update(value="", placeholder="Escribe 'siguiente' o elige una opción")

            button_row_update, user_input_update = gr.update(), gr.update(value="", interactive=False, placeholder="Elige una opción de los botones")
            btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update = make_buttons_interactive(False)
            response_text = ""

            if mode == "waiting_for_name":
                user_name = extract_name_from_greeting(message)
                current_state["user_name"], current_state["current_mode"] = user_name, "waiting_for_choice"
                response_text = f"¡Qué bueno tenerte aquí, {user_name}! Por favor, elige una opción."
                button_row_update, (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = gr.update(visible=True), make_buttons_interactive(True)
            
            elif mode == "waiting_for_author":
                search_data = search_by_author(message)
                if search_data.get("is_special_format"):
                    response_text = search_data["header"]
                    current_state["last_results"] = []
                else:
                    current_state.update({"last_results": search_data["results"], "current_page": 1, "custom_header": search_data["header"], "last_search_type": "author"})
                    response_text = format_results_page(current_state)
                current_state["current_mode"] = "waiting_for_choice"
                (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True)
                user_input_update = gr.update(value="", placeholder="Escribe 'siguiente' o elige una opción", interactive=True)
            
            elif mode == "waiting_for_topic":
                search_data = search_by_topic(message)
                current_state.update({
                    "last_results": search_data["results"], 
                    "original_query": search_data["original_query"],
                    "expanded_query": search_data["expanded_query"],
                    "current_page": 1, 
                    "last_search_type": "topic", 
                    "current_mode": "waiting_for_choice",
                    "custom_header": None
                })
                response_text = format_results_page(current_state)
                (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True)
                user_input_update = gr.update(value="", placeholder="Escribe 'siguiente' o elige una opción", interactive=True)
            
            elif mode == "waiting_for_combo_input":
                if '-' not in message:
                    response_text = "Formato incorrecto. Por favor, usa el formato: `temática - autor`. Ejemplo: `asistencia - burgos`"
                    (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True)
                    history.append({"role": "assistant", "content": response_text})
                    return history, current_state, button_row_update, btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update, gr.update(value="", interactive=True, placeholder="Inténtalo de nuevo...")

                parts = message.split('-', 1)
                topic = parts[0].strip()
                author = parts[1].strip()
                
                print(f"Iniciando búsqueda combinada (Embudo Ancho). Tema: '{topic}', Autor: '{author}'")

                search_data = search_by_topic(topic, author_filter=author, k=200)
                final_results = search_data.get("results", [])
                print(f"Búsqueda finalizada. Se encontraron {len(final_results)} documentos.")

                expanded_query = search_data.get("expanded_query", topic)
                custom_header = f"Para tu búsqueda sobre **'{topic}'** con el autor **'{author}'**, he encontrado {len(final_results)} coincidencias."
                
                current_state.update({
                    "last_results": final_results,
                    "original_query": topic,
                    "expanded_query": expanded_query,
                    "current_page": 1,
                    "custom_header": custom_header,
                    "last_search_type": "topic",
                    "current_mode": "waiting_for_choice",
                    "temp_author": None
                })
                
                response_text = format_results_page(current_state)
                (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True)
                user_input_update = gr.update(value="", placeholder="Escribe 'siguiente' o elige una opción", interactive=True)

            else:
                response_text = "Por favor, utiliza uno de los botones para continuar."
                (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True)

            history.append({"role": "assistant", "content": response_text})
            return history, current_state, button_row_update, btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update, user_input_update

        def handle_choice(history, current_state, choice):
            history.append({"role": "assistant", "content": f"Has elegido: **{choice}**"})
            mode, prompt_message = "", ""
            
            if choice == "Búsqueda por autoría": 
                mode, prompt_message = "waiting_for_author", "Por favor, escribe el nombre del autor/a o autores/as (separados por comas)."
            elif choice == "Búsqueda por temática": 
                mode, prompt_message = "waiting_for_topic", "Por favor, escribe la temática que te interesa."
            elif choice == "Búsqueda por autoría y temática": 
                mode, prompt_message = "waiting_for_combo_input", "Por favor, dime la temática y el autor(es) que buscas, separados por un guion.\n**Ejemplo:** `temática - autor(es)"

            current_state["current_mode"] = mode
            history.append({"role": "assistant", "content": prompt_message})
            b1, b2, b3, b4 = make_buttons_interactive(False)
            return history, current_state, b1, b2, b3, b4, gr.update(value="", interactive=True, placeholder="Escribe aquí...")

        def handle_bye(current_state):
            if current_state["user_name"]:
                gr.Info(f"Adiós {current_state['user_name']}, ¡recuerda que siempre estoy disponible para asistirte!")
                current_state["current_mode"] = "ended"
            
            b1, b2, b3, b4 = make_buttons_interactive(False)
            ui_input = gr.update(value="", interactive=False, placeholder="Conversación finalizada. Recarga la página para empezar de nuevo.")
            return current_state, b1, b2, b3, b4, ui_input

        user_input.submit(handle_user_input, [user_input, chatbot, chat_state], [chatbot, chat_state, button_row, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_author.click(lambda h, s: handle_choice(h, s, "Búsqueda por autoría"), [chatbot, chat_state], [chatbot, chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_topic.click(lambda h, s: handle_choice(h, s, "Búsqueda por temática"), [chatbot, chat_state], [chatbot, chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_combo.click(lambda h, s: handle_choice(h, s, "Búsqueda por autoría y temática"), [chatbot, chat_state], [chatbot, chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_bye.click(fn=handle_bye, inputs=[chat_state], outputs=[chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])

    print("\nLanzando la interfaz de CemIA con botones...")
    demo.launch(inline=True, share=False)


Lanzando la interfaz de CemIA con botones...
* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


Iniciando búsqueda combinada (Embudo Ancho). Tema: 'asistencia', Autor: 'garcia, burgos'
Búsqueda semántica por tema: 'asistencia' con filtro de autor: garcia, burgos, k=200
Término de búsqueda final: 'asistencia, asistencia escolar, inasistencia, ausentismo, tasa asistencia,  matricula,  participación,  retención'
Búsqueda finalizada. Se encontraron 4 documentos.
Iniciando búsqueda combinada (Embudo Ancho). Tema: 'asistencia', Autor: 'garcia; burgos'
Búsqueda semántica por tema: 'asistencia' con filtro de autor: garcia; burgos, k=200
Término de búsqueda final: 'asistencia, asistencia escolar, inasistencia, ausentismo, tasa asistencia,  matricula,  participación,  presentismo'
Búsqueda finalizada. Se encontraron 0 documentos.
